In [ ]:
%pwd

In [ ]:
import os 
import zipfile
from tqdm import tqdm
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import random
import onnxruntime as ort
import onnx

In [ ]:
os.chdir("../")
%pwd

In [4]:
zip_path = "data/Diddata.zip"
extract_to = "data"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    members = zip_ref.namelist()
    for member in tqdm(members, desc="Extracting"):
        zip_ref.extract(member, extract_to)

print(f"Extracted to {extract_to}")

Extracting: 100%|██████████| 725/725 [00:08<00:00, 82.50it/s] 

Extracted to data


In [5]:
base_dir = "data/Diddata"
dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
dirs.sort()  # Sort for consistent numbering

for idx, dirname in enumerate(dirs, start=1):
    src = os.path.join(base_dir, dirname)
    dst = os.path.join(base_dir, str(idx))
    os.rename(src, dst)

print(f"Renamed {len(dirs)} directories to 1-{len(dirs)}")

Renamed 55 directories to 1-55


In [6]:
# Define the SiameseNetwork class (must match your training code)
class LightweightCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(2),  

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  

            nn.Conv2d(64, 128, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2), 
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) 
        self.out_dim = 128 

    def forward(self, x):
        x = self.features(x)  
        x = self.avgpool(x) 
        x = torch.flatten(x, start_dim=1)
        return x 
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = LightweightCNN()
  
        self.project = nn.Linear(128, 512)
        self.fc = nn.Sequential(
            nn.Linear(512 * 2, 256),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(256, 1)
        )

    def forward(self, x1, x2):
        f1 = self.cnn(x1) 
        f1 = self.project(f1)  
        f2 = self.cnn(x2)  
        f2 = self.project(f2)  
        out = torch.cat([f1, f2], dim=1) 
        out = self.fc(out)
        return out.squeeze(1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = SiameseNetwork().to(device)

In [7]:
# 1. Define the same transform as in training
transform = transforms.Compose([
    transforms.Resize((112,112)),  # Resize to 112x112
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# 2. Find a real doc and selfie image
base_dir = "data/Diddata"
person_dirs = sorted([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])

doc_img_path, selfie_img_path = None, None
for person_dir in person_dirs:
    files = os.listdir(person_dir)
    doc_candidates = [os.path.join(person_dir, f) for f in files if any(x in f.lower() for x in ["id", "passport", "national"])]
    selfie_candidates = [os.path.join(person_dir, f) for f in files if "selfie" in f.lower()]
    if doc_candidates and selfie_candidates:
        doc_img_path = doc_candidates[0]
        selfie_img_path = selfie_candidates[0]
        break

assert doc_img_path is not None and selfie_img_path is not None, "No doc/selfie pair found!"

# 3. Load and preprocess images
doc_img = Image.open(doc_img_path).convert("RGB")
selfie_img = Image.open(selfie_img_path).convert("RGB")
doc_tensor = transform(doc_img).unsqueeze(0).to(device)     
selfie_tensor = transform(selfie_img).unsqueeze(0).to(device)

# 4. Load model and export to ONNX
model.load_state_dict(torch.load("Model/best_kyc_siamese.pt", map_location=device))
model.eval()

class ExportableSiameseNetwork(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, x1, x2):
        logits = self.base_model(x1, x2)
        return torch.sigmoid(logits)  # Adds sigmoid to get probability

# 4. Create exportable model
export_model = ExportableSiameseNetwork(model).to(device)

onnx_path = "Model/best_kyc_siamese.onnx"
torch.onnx.export(
    export_model,
    (doc_tensor, selfie_tensor),
    onnx_path,
    export_params=True,
    do_constant_folding=True,
    input_names=["doc_img", "selfie_img"],
    output_names=["output"],
    dynamic_axes={
        "doc_img": {0: "batch_size"},
        "selfie_img": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=12
)

print(f"Model exported to {onnx_path} using real images:\nDoc: {doc_img_path}\nSelfie: {selfie_img_path}")

Model exported to Model/best_kyc_siamese.onnx using real images:
Doc: data/Diddata/1/ID_1.jpg
Selfie: data/Diddata/1/Selfie_8.jpg


In [8]:
# 1. Define the same transform as in training
transform = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [13]:

base_dir = "data/Diddata"
person_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
random.shuffle(person_dirs)

pair_type = random.choice(["positive", "negative"])

if pair_type == "positive":
    # Positive: doc and selfie from the same person
    for person_dir in person_dirs:
        files = os.listdir(person_dir)
        doc_candidates = [os.path.join(person_dir, f) for f in files if any(x in f.lower() for x in ["id", "passport", "national"])]
        selfie_candidates = [os.path.join(person_dir, f) for f in files if "selfie" in f.lower()]
        if doc_candidates and selfie_candidates:
            doc_img_path = random.choice(doc_candidates)
            selfie_img_path = random.choice(selfie_candidates)
            break
    label = 1
else:
    # Negative: doc from one person, selfie from another
    doc_img_path, selfie_img_path = None, None
    while True:
        person1, person2 = random.sample(person_dirs, 2)
        files1 = os.listdir(person1)
        files2 = os.listdir(person2)
        doc_candidates = [os.path.join(person1, f) for f in files1 if any(x in f.lower() for x in ["id", "passport", "national"])]
        selfie_candidates = [os.path.join(person2, f) for f in files2 if "selfie" in f.lower()]
        if doc_candidates and selfie_candidates:
            doc_img_path = random.choice(doc_candidates)
            selfie_img_path = random.choice(selfie_candidates)
            break
    label = 0

assert doc_img_path is not None and selfie_img_path is not None, "No valid doc/selfie pair found!"

# Preprocess images
doc_img = Image.open(doc_img_path).convert("RGB")
selfie_img = Image.open(selfie_img_path).convert("RGB")
doc_tensor = transform(doc_img).unsqueeze(0).cpu().numpy()
selfie_tensor = transform(selfie_img).unsqueeze(0).cpu().numpy()


In [14]:
# 4. Run ONNX inference

ort_session = ort.InferenceSession("Model/best_kyc_siamese.onnx")
outputs = ort_session.run(
    None,
    {"doc_img": doc_tensor, "selfie_img": selfie_tensor}
)
onnx_prob = float(outputs[0].squeeze())  
print(f"Pair type: {'Positive (same person)' if label==1 else 'Negative (different people)'}")
print(f"Random doc: {doc_img_path}")
print(f"Random selfie: {selfie_img_path}")
print("ONNX output (probability): {:.2f}".format(onnx_prob))

Pair type: Positive (same person)
Random doc: data/Diddata/4/ID_2.jpg
Random selfie: data/Diddata/4/Selfie_10.jpg
ONNX output (probability): 0.97


In [15]:
# Use actual inference images from the inference directory for ONNX model verification

# 1. Define the same transform as in training
transform = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# 2. Set the inference directory and image paths
inference_dir = "inference"
doc_img_path = os.path.join(inference_dir, "ID-2.jpeg")
selfie_img_path = os.path.join(inference_dir, "Selfie.jpg")

assert os.path.exists(doc_img_path), f"Doc image not found: {doc_img_path}"
assert os.path.exists(selfie_img_path), f"Selfie image not found: {selfie_img_path}"

# 3. Load and preprocess images
doc_img = Image.open(doc_img_path).convert("RGB")
selfie_img = Image.open(selfie_img_path).convert("RGB")
doc_tensor = transform(doc_img).unsqueeze(0).cpu().numpy()
selfie_tensor = transform(selfie_img).unsqueeze(0).cpu().numpy()

# 4. Run ONNX inference
ort_session = ort.InferenceSession("Model/best_kyc_siamese.onnx")
outputs = ort_session.run(
    None,
    {"doc_img": doc_tensor, "selfie_img": selfie_tensor}
)
onnx_prob = float(outputs[0].squeeze())
print(f"Doc image: {doc_img_path}")
print(f"Selfie image: {selfie_img_path}")
print("ONNX output (probability): {:.3f}".format(onnx_prob))

Doc image: inference/ID-2.jpeg
Selfie image: inference/Selfie.jpg
ONNX output (probability): 0.876


In [17]:
# Make negative pairs using the inference doc image and random selfie images from the dataset

import os
import random
from PIL import Image
import torch
from torchvision import transforms
import onnxruntime as ort
import numpy as np

# 1. Define the same transform as in training
transform = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# 2. Set the inference doc image path
inference_dir = "inference"
doc_img_path = os.path.join(inference_dir, "ID-2.jpeg")
assert os.path.exists(doc_img_path), f"Doc image not found: {doc_img_path}"

# 3. Find random selfie images from other people in the dataset
base_dir = "data/Diddata"
person_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
negative_selfies = []

for person_dir in person_dirs:
    selfie_candidates = [os.path.join(person_dir, f) for f in os.listdir(person_dir) if "selfie" in f.lower()]
    if selfie_candidates:
        negative_selfies.extend(selfie_candidates)

# Remove the inference selfie if present
inference_selfie_path = os.path.join(inference_dir, "Selfie.jpg")
negative_selfies = [s for s in negative_selfies if os.path.abspath(s) != os.path.abspath(inference_selfie_path)]

# 4. Pick a few random negative selfie images and run ONNX inference
num_negatives = 3  # Number of negative pairs to test
random.shuffle(negative_selfies)
selected_selfies = negative_selfies[:num_negatives]

doc_img = Image.open(doc_img_path).convert("RGB")
doc_tensor = transform(doc_img).unsqueeze(0).cpu().numpy()

ort_session = ort.InferenceSession("Model/best_kyc_siamese.onnx")

for idx, selfie_path in enumerate(selected_selfies, 1):
    selfie_img = Image.open(selfie_path).convert("RGB")
    selfie_tensor = transform(selfie_img).unsqueeze(0).cpu().numpy()
    outputs = ort_session.run(
        None,
        {"doc_img": doc_tensor, "selfie_img": selfie_tensor}
    )
    onnx_prob = float(outputs[0].squeeze())
    print(f"Negative Pair {idx}:")
    print(f"  Doc image: {doc_img_path}")
    print(f"  Selfie image: {selfie_path}")
    print(f"  ONNX output (probability): {onnx_prob:.3f}\n")

Negative Pair 1:
  Doc image: inference/ID-2.jpeg
  Selfie image: data/Diddata/30/Selfie_11.jpg
  ONNX output (probability): 0.238

Negative Pair 2:
  Doc image: inference/ID-2.jpeg
  Selfie image: data/Diddata/21/Selfie_10.jpg
  ONNX output (probability): 0.975

Negative Pair 3:
  Doc image: inference/ID-2.jpeg
  Selfie image: data/Diddata/23/Selfie_6.jpg
  ONNX output (probability): 0.912



In [18]:

model = onnx.load("Model/best_kyc_siamese.onnx")

print(f"Model IR version: {model.ir_version}")
print(f"Producer: {model.producer_name} v{model.producer_version}")
print(f"Graph name: {model.graph.name}")
print(f"Inputs:")
for i in model.graph.input:
    print(f" - {i.name}, shape: {[d.dim_value for d in i.type.tensor_type.shape.dim]}")

print(f"\nOutputs:")
for o in model.graph.output:
    print(f" - {o.name}, shape: {[d.dim_value for d in o.type.tensor_type.shape.dim]}")

print(f"\nTotal nodes: {len(model.graph.node)}")
print(f"Unique node types: {set(n.op_type for n in model.graph.node)}")


Model IR version: 7
Producer: pytorch v2.7.0
Graph name: main_graph
Inputs:
 - doc_img, shape: [0, 3, 112, 112]
 - selfie_img, shape: [0, 3, 112, 112]

Outputs:
 - output, shape: [0]

Total nodes: 36
Unique node types: {'Gemm', 'Relu', 'Concat', 'MaxPool', 'Sigmoid', 'Flatten', 'Squeeze', 'Conv', 'GlobalAveragePool'}
